In [1]:
import org.apache.spark.SparkConf
import org.apache.spark.sql.SparkSession
import sys.process._

In [2]:
val conf = new SparkConf()

conf = org.apache.spark.SparkConf@6dc8adb8


org.apache.spark.SparkConf@6dc8adb8

In [3]:
val spark = SparkSession.builder.appName("LABA5").config(conf).getOrCreate()

spark = org.apache.spark.sql.SparkSession@3e9e2535


org.apache.spark.sql.SparkSession@3e9e2535

In [4]:
"hdfs dfs -ls /labs/slaba05/" !

Found 2 items
-rw-r--r--   3 hdfs hdfs   24271415 2022-01-06 18:46 /labs/slaba05/lab05_test.csv
-rw-r--r--   3 hdfs hdfs  176160768 2022-01-06 18:46 /labs/slaba05/lab05_train.csv


0

In [5]:
val df_train_raw = spark.read.format("csv").option("dilimeter", ",").option("inferSchema", "true").option("header", "true").load("/labs/slaba05/lab05_train.csv")

df_train_raw = [_c0: int, ID: int ... 115 more fields]


[_c0: int, ID: int ... 115 more fields]

In [6]:
val df_test_raw = spark.read.format("csv").option("inferSchema", "true").option("dilimeter", ",").option("header", "true").load("/labs/slaba05/lab05_test.csv")

df_test_raw = [_c0: int, ID: int ... 114 more fields]


[_c0: int, ID: int ... 114 more fields]

In [7]:
import org.apache.spark.sql.functions.udf
import org.apache.spark.sql.functions.col
import org.apache.spark.ml.feature.OneHotEncoder
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.classification.GBTClassifier
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.linalg.Vector

In [8]:
val ageToGroup = udf((age: Int) => if (age < 288) 0 else if (age < 408) 1 else if (age < 528) 2 else if (age < 648) 3 else 4)

ageToGroup = UserDefinedFunction(<function1>,IntegerType,Some(List(IntegerType)))


UserDefinedFunction(<function1>,IntegerType,Some(List(IntegerType)))

In [9]:
val df_train = df_train_raw.withColumn("AGE_GROUP", ageToGroup(col("AGE")))

df_train = [_c0: int, ID: int ... 116 more fields]


[_c0: int, ID: int ... 116 more fields]

In [10]:
val df_test = df_test_raw.withColumn("AGE_GROUP", ageToGroup(col("AGE")))

df_test = [_c0: int, ID: int ... 115 more fields]


[_c0: int, ID: int ... 115 more fields]

In [11]:
val ohEncoder = new OneHotEncoder().setDropLast(true).setInputCol("AGE_GROUP").setOutputCol("AGE_VEC")

ohEncoder = oneHot_5538bcc52672


oneHot_5538bcc52672

In [12]:
val assembler = new VectorAssembler().setInputCols(Array("CR_PROD_CNT_IL", "TURNOVER_DYNAMIC_IL_1M", "REST_DYNAMIC_FDEP_1M", "REST_DYNAMIC_SAVE_3M",
        "CR_PROD_CNT_VCU", "REST_AVG_CUR", "CR_PROD_CNT_TOVR", "CR_PROD_CNT_PIL", "TURNOVER_CC", "TURNOVER_PAYM", "CR_PROD_CNT_CC", "REST_DYNAMIC_FDEP_3M",
        "REST_DYNAMIC_IL_1M", "CR_PROD_CNT_CCFP", "REST_DYNAMIC_CUR_1M", "REST_AVG_PAYM", "LDEAL_GRACE_DAYS_PCT_MED", "REST_DYNAMIC_CUR_3M", "TURNOVER_DYNAMIC_CUR_1M",
        "REST_DYNAMIC_PAYM_3M", "REST_DYNAMIC_IL_3M", "TURNOVER_DYNAMIC_IL_3M", "REST_DYNAMIC_PAYM_1M", "TURNOVER_DYNAMIC_CUR_3M", "CLNT_SETUP_TENOR",
        "TURNOVER_DYNAMIC_PAYM_3M",  "TURNOVER_DYNAMIC_PAYM_1M", "REST_DYNAMIC_CC_1M", "TURNOVER_DYNAMIC_CC_1M", "REST_DYNAMIC_CC_3M", "TURNOVER_DYNAMIC_CC_3M", "AGE_VEC"
    )).setOutputCol("features")

assembler = vecAssembler_2dbd7f4885e2


vecAssembler_2dbd7f4885e2

In [13]:
val gbt = new GBTClassifier().setFeaturesCol("features").setLabelCol("TARGET")

gbt = gbtc_3e69ab984f54


gbtc_3e69ab984f54

In [14]:
val pipeline = new Pipeline().setStages(Array(ohEncoder, assembler, gbt))

pipeline = pipeline_db14f3b3188b


pipeline_db14f3b3188b

In [15]:
val df_train_clear = df_train.filter(col("TURNOVER_CC").isNotNull)

df_train_clear = [_c0: int, ID: int ... 116 more fields]


[_c0: int, ID: int ... 116 more fields]

In [16]:
val model = pipeline.fit(df_train_clear)

model = pipeline_db14f3b3188b


pipeline_db14f3b3188b

In [17]:
val predictions = model.transform(df_test)

predictions = [_c0: int, ID: int ... 120 more fields]


[_c0: int, ID: int ... 120 more fields]

In [18]:
val getProb = udf((prob: Vector) => prob(1))

getProb = UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))


UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))

In [19]:
val res = predictions.withColumn("prob", getProb(col("probability"))).select(col("ID").as("id"), col("prob").as("target")).repartition(1)

res = [id: int, target: double]


[id: int, target: double]

In [20]:
"hdfs dfs -ls lab05.csv" !

Found 2 items
-rw-r--r--   3 igor.gorchakov igor.gorchakov          0 2022-11-07 21:02 lab05.csv/_SUCCESS
-rw-r--r--   3 igor.gorchakov igor.gorchakov    1195350 2022-11-07 21:02 lab05.csv/part-00000-fd7979e4-abaf-4c74-a35e-926e93f5675e-c000.csv


0

In [24]:
"hdfs dfs -copyToLocal lab05.csv/part-00000-fd7979e4-abaf-4c74-a35e-926e93f5675e-c000.csv lab05.csv" !

0

In [25]:
"ls -l" !

total 1182632
-rw-rw-r-- 1 igor.gorchakov igor.gorchakov      1089 Oct 11 19:34 03 Spark DataFrame.ipynb
-rw-r--r-- 1 igor.gorchakov igor.gorchakov  69519728 Oct 12 23:25 DO_record_per_line.json
-rw-rw-r-- 1 igor.gorchakov igor.gorchakov      8884 Oct  8 11:04 first sc_SparkContext.ipynb
-rw-rw-r-- 1 igor.gorchakov igor.gorchakov       162 Oct 12 20:57 lab01.json
-rw-rw-r-- 1 igor.gorchakov igor.gorchakov       162 Oct 23 17:05 lab01_ttt.json
-rw-rw-r-- 1 igor.gorchakov igor.gorchakov       874 Nov  1 14:11 lab02.json
-rw-rw-r-- 1 igor.gorchakov igor.gorchakov  71530040 Nov  4 20:08 lab03.csv
-rw-r--r-- 1 igor.gorchakov igor.gorchakov   1195350 Nov  7 21:15 lab05_.csv
-rw-r--r-- 1 igor.gorchakov igor.gorchakov   1195350 Nov  8 22:03 lab05.csv
-rw-rw-r-- 1 igor.gorchakov igor.gorchakov     17057 Nov  8 22:02 lab05.ipynb
-rw-rw-r-- 1 igor.gorchakov igor.gorchakov    119969 Nov  8 21:48 lab05 tg buddy.ipynb
-rw-rw-r-- 1 igor.gorchakov igor.gorchakov     10022 Oct 30 17:21 laba01.ipynb
-rw

0

In [23]:
// spark.stop()

Name: Syntax Error.
Message: 
StackTrace: 